# Clustering - Fixed Window - DL extractor
- Use Deep Learning based extractors to get abstract features by giving the entire detection subseq as input
- Use these features to cluster the detections with similar anomalies
- The feature extraction is NOT dependent on the corresponding normal behaviour subtrace
- This will serve as benchmark peroformance with off the shelf models, without any optimization 
- We tested this approach across all applications

In [ ]:
'''
TODO:
- select a DL based feature extractor
- check if we need to fine-tune the feature extractor on training data before using it on test data
- make a list of suitable clustering algorithms for different usecases

CLUSTERING
- use the feature extractor to extract features from the detections
- try to cluster the detections based on the extracted features

Steps:
- try TSFEDL for anomaly detection using sepcialized anomaly detectors (detect)
 
- try TSFEDL to extract features using SOTA DL models (diag)
- try TSFEL to extract handcrafted features (statistical, spectral, temporal, etc) (diag)
'''
